# Preprocess CBH forcing files, solar geometry output, and climate adjustments.

This notebook demonstrates  
1) Conversion of PRMS CBH files to NetCDF for use with pywatershed,
2) Writing out variables of solar geometry
3) Applying parameter adjustments to get static atmospheric forcings used by the hydrology. If you are not varying parameters to PRMSAtmosphere, your atmosphere does not change between runs and this can save considerable time. We show how to do this with solar variables on file (from step 2) or using a PRMSSolarGeom instance. 

This notebook assumes you have an editable install of pywatershed (`pip install -e .` from the root of
the cloned repository), to get necessary domain information.

In [ ]:
# auto-format the code in this notebook
%load_ext jupyter_black

In [ ]:
from copy import deepcopy
import pathlib as pl
from pprint import pprint
import shutil

import numpy as np
import pywatershed as pws
import xarray as xr

pws_repo_root = pws.constants.__pywatershed_root__.parent

In [ ]:
nb_output_dir = pl.Path("./preprocess_cbh_adj")
if nb_output_dir.exists():
    shutil.rmtree(nb_output_dir)
nb_output_dir.mkdir()

# This works with domains in the pywatershed repository
# you can configure for your domain.
dom_name = "drb_2yr"
dom_dir = pws_repo_root / f"test_data/{dom_name}"
param_file = dom_dir / "myparam.param"
control_file = dom_dir / "control.test"
dis_file = dom_dir / "parameters_dis_hru.nc"

## Convert CBH files to netcdf

In [ ]:
params = pws.parameters.PrmsParameters.load(param_file)

cbh_files = {
    "prcp": dom_dir / "prcp.cbh",
    "tmax": dom_dir / "tmax.cbh",
    "tmin": dom_dir / "tmin.cbh",
}

cbh_dir = nb_output_dir / f"cbh"
cbh_dir.mkdir(exist_ok=True)

for kk, vv in cbh_files.items():
    out_file = cbh_dir / f"{kk}.nc"
    pws.utils.cbh_file_to_netcdf({kk: vv}, params, out_file)

## Write solar geometry files

In [ ]:
solar_geom_dir = nb_output_dir / "solar_geom"
solar_geom_dir.mkdir(exist_ok=True)

solar_geom_output_vars = ["soltab_horad_potsw", "soltab_potsw"]

control = pws.Control.load_prms(control_file, warn_unused_options=False)

# FIX this not working
# control.options = control.options | {
#     "netcdf_output_dir": solar_geom_dir,
#     "netcdf_output_var_names": [
#         "soltab_horad_potsw",
#         "soltab_potsw",
#     ],
# }

solar_geom = pws.PRMSSolarGeometry(
    control,
    None,
    params,
    # netcdf_output_dir=solar_geom_dir,
    # netcdf_output_vars=solar_geom_output_vars,  ## this is apparently not working either
)
solar_geom.initialize_netcdf(
    output_dir=solar_geom_dir,
    # output_vars=solar_geom_output_vars,  ## this is apparently not working either, no output
)
control.advance()
solar_geom.advance()
solar_geom.output()
# why is it outputing soltab_sunhrs?

In [ ]:
var = "soltab_potsw"
da = xr.open_dataarray(solar_geom_dir / f"{var}.nc", decode_timedelta=False)
display(da)
print(da[-1, 0:100].values)
da.close()

## Preprocess atmospheric forcings without solar geometry files present

When a `PRMSAtmosphere` object is initalized with a `netcdf_output_dir` argument, the adjusted forcings 
are written to this location. Unless one requests specific variables only, all variables are written. 

Typically, the `soltab_potsw.nc` and `soltab_horad_potsw.nc` input files are not available as inputs. 
(These are only output in a fixed width format by a version of PRMS5.2.1 in the pynhm repository
that is translated to netCDF when setting up test data). Here we show how to get the CBH adjustments
to output files using PRMSSolarGeometry instead of soltab files. The next section will show how to use available soltab files we created above.

In [ ]:
cbh_files_dict = {ff.with_suffix("").name: ff for ff in cbh_dir.glob("*.nc")}

In [ ]:
atm_dir = nb_output_dir / "atm_without_solar_files"
atm_dir.mkdir(exist_ok=True)

control = pws.Control.load_prms(control_file, warn_unused_options=False)
solar_geom = pws.PRMSSolarGeometry(control, None, params)

atm = pws.PRMSAtmosphere(
    control,
    None,
    params,
    **cbh_files_dict,
    soltab_horad_potsw=solar_geom.soltab_horad_potsw,
    soltab_potsw=solar_geom.soltab_potsw,
)
atm.initialize_netcdf(atm_dir)
control.advance()
solar_geom.advance()
atm.advance()
atm.calculate(1)
atm.output()

In [ ]:
var = "potet"
da = xr.open_dataarray(atm_dir / f"{var}.nc")
display(da)
print(da[-1, 0:100].values)
da.close()

## Preprocess atmospheric forcings with solar geometry files present
We repeat the above, dropping the `PRMSSolarGeometry` object as its information is now coming from the soltab files. 

In [ ]:
cbh_files_dict = {ff.with_suffix("").name: ff for ff in cbh_dir.glob("*.nc")}
solar_files_dict = {
    ff.with_suffix("").name: ff for ff in solar_geom_dir.glob("*.nc")
}
del solar_files_dict["soltab_sunhrs"]
atm_input_files_dict = cbh_files_dict | solar_files_dict

In [ ]:
atm_solar_files_dir = nb_output_dir / "atm_without_solar_files"
atm_solar_files_dir.mkdir(exist_ok=True)

control = pws.Control.load_prms(control_file, warn_unused_options=False)
solar_geom = pws.PRMSSolarGeometry(control, None, params)

atm = pws.PRMSAtmosphere(
    control,
    None,
    params,
    **atm_input_files_dict,
)
atm.initialize_netcdf(atm_solar_files_dir)
control.advance()
atm.advance()
atm.calculate(1)
atm.output()

In [ ]:
var = "potet"
da = xr.open_dataarray(atm_dir / f"{var}.nc")
display(da)
print(da[-1, 0:100].values)
da.close()